In [2]:
import tiktoken
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


In [3]:
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [4]:
loader = PyPDFLoader("/Users/jaehyeongpark/Documents/kb_report.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function = tiktoken_len)
docs = text_splitter.split_documents(pages)


model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
ko = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/opt/homebrew/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
db = FAISS.from_documents(docs, ko)

In [6]:
query = "주택 시장 동향은 어떻게 되어있나요?"
docs = db.similarity_search(query)
print(docs[0].page_content)

 서울과 수도권  아파트 를 중심으로  전세가격  상승세가  지속되고  있으며 , 수도권  전세가격전망지수는  
지난해  11월 이후 2개월 만에 다시 ‘상승 전망’으로 전환 
∙ 아파트  선호 현상과 올해 아파트  입주물량  감소 등의 영향으로  당분간  수도권  아파트  전세가
격 상승세가  지속될  전망 
 
그림 7| 유형별  주택 전세가격지수  변동률  추이  그림 8| 지역별  주택 전세가격전망지수  추이 
  
 
자료: KB국민은행   자료: KB국민은행


In [7]:
docs_and_scores = db.similarity_search_with_score(query)
docs_and_scores

[(Document(page_content='\uf06d 서울과 수도권  아파트 를 중심으로  전세가격  상승세가  지속되고  있으며 , 수도권  전세가격전망지수는  \n지난해  11월 이후 2개월 만에 다시 ‘상승 전망’으로 전환 \n∙ 아파트  선호 현상과 올해 아파트  입주물량  감소 등의 영향으로  당분간  수도권  아파트  전세가\n격 상승세가  지속될  전망 \n \n그림 7| 유형별  주택 전세가격지수  변동률  추이  그림 8| 지역별  주택 전세가격전망지수  추이 \n  \n \n자료: KB국민은행   자료: KB국민은행', metadata={'source': '/Users/jaehyeongpark/Documents/kb_report.pdf', 'page': 4}),
  0.75258374),
 (Document(page_content='\uf06d 1월 주택 매매가격은  전월 대비 하락폭이  확대되었으나  지난해  월평균  하락폭  대비 3분의 1 \n수준이며 , 하락폭이  가장 큰 대구도  지난해  월평균  하락폭  대비 절반 수준에 그침 \n\uf06d 주택 매매가격전망지수는  여전히  ‘상승 전망’보다 ‘하락 전망’ 비중이  크지만  연초 이후 정부의  규제 \n완화와  4월 총선, 금리 인하에 대한 기대감  등으로  하락폭이  확대될  가능성은  적음 \n \n그림 3| 지역별  주택 매매가격지수  변동률   그림 4| 지역별  주택 매매가격전망지수  추이 \n  \n \n자료: KB국민은행   자료: KB국민은행', metadata={'source': '/Users/jaehyeongpark/Documents/kb_report.pdf', 'page': 3}),
  0.75613165),
 (Document(page_content='5 \n[주택 전세] 수도권  아파트를 중심으로  전세가격  상승세가 지속될 전망 \n \n■ 주택 전세가격은 지난해  9월 이후 5개월 연속 상승세를 이어감  \n\uf06d 1월 전국 주택 전세가격 은 전

In [8]:
db.save_local("faiss_index")

In [9]:
new_db = FAISS.load_local("faiss_index", ko)

query = "주택담보대출 동향은?"
docs = new_db.similarity_search_with_relevance_scores(query, k=3, fetch_k=10, lambda_mult=0.3)


print("질문 : {} \n".format(query))
for i in range(len(docs)):
    print("{0}번쨰 유사 문서 유사도 \n{1}" .format(i+1, round(docs[i][1],2)))
    print("-"*100)
    print(docs[i][0].page_content)
    print("\n")
    print(docs[i][0].metadata)
    print("-"*100)

질문 : 주택담보대출 동향은? 

1번쨰 유사 문서 유사도 
0.59
----------------------------------------------------------------------------------------------------
9 
[주택 금융] 주택담보대출  증가세가 소폭 둔화 
 
■ 2023년 12월 은행권  주택담보대출 (전세자금대출  포함)은 전월 대비 5.2조 원 증가 
 8월 전월 대비 7조 원 증가하며 최고치를  기록한  후 전반적으로  증가세 가 둔화되고  있으나 , 여전히  
2022 년 월평균  순증액 (1.7조 원)을 크게 상회 
∙ 전세자금대출  잔액은  2023 년 6월 이후 큰 변화 없이 보합세를  유지 
 거래 침체와  정부의  가계대출  관리 기조 등을 감안할  때 주택담보대출  증가세가  한풀 꺾일 것으로  전망 
 
그림 20| 은행권  주택담보대출  규모 추이  그림 21| 은행권  전세자금대출  규모 추이 
  
 
주: 전세자금대출  등 주택 관련 대출 포함 
자료: 한국은행   주: 월간 순증액은  한국은행  월간 수보 기준, 기금재원대출  제외 
자료: 한국은행


{'source': '/Users/jaehyeongpark/Documents/kb_report.pdf', 'page': 8}
----------------------------------------------------------------------------------------------------
2번쨰 유사 문서 유사도 
0.53
----------------------------------------------------------------------------------------------------
자료: 한국은행  
 
■ 주택담보대출 (신규 취급액  기준) 금리는 2개월 연속 하락세를 이어감  
 2023 년 12월 신규 취급액  기준 주택담보대출  금리는  4.16% 로 전월 대비 0.32%p 하락 
∙ 